In [29]:
import pandas as pd
import numpy as np
import re
import os
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

In [30]:
def get_dicts_of_mode_files(dir_name):
    """Возвращает словари файлов с режимами для обучения и для валидации

    Keyword arguments:
    dir_name -- путь к папке в которой лежат файлы с режимами
    
    Возвращает: два словаря датафреймов
    """
    file_names_42 = os.listdir(dir_name+'/42')
    file_names_28 = os.listdir(dir_name+'/28')
    # файлы для валидации в словарь 
    valid_files = dict()
    for name in file_names_28:
        valid_files[name] = pd.read_excel(dir_name+'/28'+'\\'+name,skiprows=1)
        
    # файлы для обучения в словарь 
    mode_files = dict()
    for name in file_names_42:
        mode_files[name] = pd.read_excel(dir_name+'/42'+'\\'+name,skiprows=1)
    # Удаление лишнего столбца с комментариями из файла
    del mode_files['2014.xls']['Unnamed: 42']
    return [mode_files, valid_files]

In [31]:
def rep_part(ls):
    """Удаляет .0 в номере партии и возвращает список без него"""
    for x in ls:
        x = str(x)
        x = x.replace('.0', '')
    return ls

In [32]:
def gen_batch_number_list(files_dict):
    """Создает список номеров партий из правой части таблицы

    Keyword arguments:
    files_dict -- словарь датафреймов, из которых брать номера партий (ключ в словаре - нназвание файла)
    
    Возвращает: list номеров партий
    """
    batch_number_list = []
    for mode in files_dict.values():
        for x in mode['№ партии.1']:
            batch_number_list.append(str(x).replace('.0', ''))

    batch_number_list = rep_part(batch_number_list)
    print(len(batch_number_list))
    return batch_number_list

In [33]:
def print_file_sizes(mode_files, valid_files):
    # выводим размерности всех файлов
    print('train:')
    for i in mode_files.values():
        print(i.shape)
    print('validation:')
    for i in valid_files.values():
        print(i.shape)

In [34]:
mode_files, valid_files = get_dicts_of_mode_files('DATA/source/modes')

In [35]:
# mode_files.shape, valid_files.shape

In [36]:
def clean_dict(files_dict):
    """Убирает заголовки у датафреймов из словаря, удаляет лишние столбцы

    Keyword arguments:
    files_dict -- словарь файлов (ключ в словаре - нназвание файла)
    
    Возвращает: list номеров партий
    """
    # убираем заголовки
    tmp = dict()
    for name, mode in zip(files_dict.keys(), files_dict.values()):
        tmp[name] = mode.copy()
        tmp[name].columns = range(mode.shape[1])       
   
    return tmp

In [37]:
def return_headers(df, headers_list, batch_list):
    """Возвращаем заголовки и номера партий, удаляем партии без номеров"""
    df.columns = headers_list[:len(df.columns)]
    df['№ партии'] = batch_list
    df = df[df['№ партии'] != 'nan']
    print(df.shape)
    return df

In [38]:
def get_df_modes(dict_modes, headers_file_name):
    # Сохраняем столбец с номером партии в переменную batch_number_list 
    batch_number_list = gen_batch_number_list(dict_modes)
    
     # убираем последние пять столбцов из всех таблиц
    for mode in dict_modes.values():
        mode.drop(list(mode.columns[-5:-2])+[mode.columns[-1]], axis=1, inplace=True)
    
    # Сохраняем заголовки
    header = dict_modes[headers_file_name].columns
    # удаляем заголовки и лишние столбцы
    dict_modes_tmp_copy = clean_dict(dict_modes)
    df_modes = pd.DataFrame()
    # слияние всех режимов
    for df in dict_modes_tmp_copy.values():
        df_modes = pd.concat([df_modes,df])
    # Сбрасываем индекс, чтобы не было повторяющихся
    df_modes.reset_index(drop=True, inplace=True)
#     print(df_modes.shape)
    # Возвращаем заголовки и номера партий, удаляем партии без номеров
    df_modes = return_headers(df_modes, header, batch_number_list)
#     print(df_modes.shape)
    return df_modes

In [39]:
headers_for_42_col_files = '2017_1и2кварталы.xls'
df_modes = get_df_modes(mode_files, headers_for_42_col_files)

headers_for_28_col_files = '2018_01.xls'
df_valid = get_df_modes(valid_files, headers_for_28_col_files)

129559
(129086, 38)
8989
(8958, 24)


In [40]:
# df_modes.shape, df_valid.shape

In [41]:
# df_modes.columns, df_valid.columns

In [42]:
# исходная таблица без пакетов
df_modes = df_modes.drop(['№ пакета'], axis=1)
df_valid = df_valid.drop(['№ пакета'], axis=1)

In [43]:
# Удаляем партии без номера плавки
df_modes = df_modes[df_modes['плавка'] != 'nan']
print(df_modes.shape)
df_valid = df_valid[df_valid['плавка'] != 'nan']
df_valid.shape

(129086, 37)


(8958, 23)

In [44]:
# Удаляем строки со сбитым заполнением 
str_to_del = [3.75, 4.7, 4.18, 4.85, 4.78, 4.56, 3.71, 4.7, 4.63, 4.65, 4.52, 1.57, 3.87, 3.89, 2.5, 1.91, 2.98, 
                  4.6, 4.71, 2.36, 4.39, 4.68, 1.82, 2.74, 2.81, 3.61]


for i in str_to_del:
    df_modes = df_modes[df_modes['НД (гост, ту)'] != i]

df_modes.shape

(129047, 37)

In [45]:
# Разносим шаг на закалочной и отпускной печах в разные столбцы и очищаем их
def clean_steps(x):
    x = str(x)
    x = x.replace(u'X', '/')
    x = x.replace(u'x', '/')
    x = x.replace(u'Х', '/')
    x = x.replace(u'х', '/')
    x = x.replace(',', '.')
    x = x.replace('\\', '/')
    x = x.replace('/-', '/0')
    x = x.replace('-/', '0/')
    x = x.replace('***', '0')
    x = x.replace('*', '0')
    x = x.replace('-', '/')
    x = x.replace('nan', '')
    return x

def convert_step0(x):
    return clean_steps(x).split('/')[0]
   
def convert_step1(x):
    try:
        return clean_steps(x).split('/')[1]
    except:
        return x

# df_modes = df_modes[df_modes[u'шаг балок закалочная/отпускная печи, сек']!='***']
df_modes[u'шаг балок закалочная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
df_modes[u'шаг балок отпускная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)

df_valid[u'шаг балок закалочная печь, сек'] = df_valid[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
df_valid[u'шаг балок отпускная печь, сек'] = df_valid[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)
print(df_modes.shape, df_valid.shape)

(129047, 39) (8958, 25)


In [46]:
# Разделяем диаметр и толщину стенки и чистим их
def split_d(x):
    x = str(x)
    x = x.replace(',', '.')
    return re.split('х|Х|x|X', x)[0]

def split_s(x):
    x = str(x)
    x = x.replace(',', '.')
    try:
        return re.split('х|Х|x|X', x)[1]
    except:
        return x
    
def clean_d_s(x):
    x = str(x)
    x = x.replace('п', '')
    x = x.replace('П', '')
    x = x.replace('nan', '')
    x = x.replace(',', '.')
    return x

df_modes['диаметр'] = df_modes[u'D/S'].apply(split_d)
df_modes['толщина стенки'] = df_modes[u'D/S'].apply(split_s)
df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(clean_d_s)
df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
df_modes['диаметр'] = df_modes['диаметр'].apply(clean_d_s)

df_valid['диаметр'] = df_valid[u'D/S'].apply(split_d)
df_valid['толщина стенки'] = df_valid[u'D/S'].apply(split_s)
df_valid['толщина стенки'] = df_valid['толщина стенки'].apply(clean_d_s)
df_valid['толщина стенки'] = df_valid['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
df_valid['диаметр'] = df_valid['диаметр'].apply(clean_d_s)

In [47]:
# Чистим Скорость прохождения трубы через спрейер, м/с
def clean_u_sp(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u'ек', '')
    x = x.replace(u'\мс', '')
    x = x.replace(u'm/c', '')
    x = x.replace(u'М/С', '')
    x = x.replace(u'М\С', '')
    x = x.replace(u'nan', '')
    x = x.replace(u'м/с', '')
    x = x.replace(u'м\с', '')
    x = x.replace(u',', u'.')
    x = x.replace(u'м\\', '')
    x = x.replace(u'м/', '')
    x = x.replace(u'*****', u'')
    x = x.replace(u'****', u'')
    x = x.replace(u'***', u'')
    x = x.replace(u'29.03.2017', u'')
    x = x.replace(',','.')
    x = x.replace('041','0.41')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')  
    x = x.replace('м','')
    x = x.replace('021', '0.21')
    x = x.replace('1.8', '0.18')
    x = x.replace('2.0', '0.2')
    x = x.replace('012', '0.12')
    if x == '45':
        return '0.45'
    if x == '064':
        return '0.64'
    x = x.replace(u'0.7.', '0.7')
    try:
        y = x.split('-')
        x = (float(y[0])+float(y[1]))/2
    except:
        pass
    return x
    
col = 'Скорость прохождения трубы через спрейер, м/с' 
df_modes[col] = df_modes[col].apply(clean_u_sp)
df_valid[col] = df_valid[col].apply(clean_u_sp)
print(df_modes.shape, df_valid.shape)

(129047, 41) (8958, 27)


In [48]:
# Чистим время выдержки в закалочной ванне, сек.
def clean_time_bath(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u' ', '')
    x = x.replace(u'.', '')
    x = x.replace(u'J55', '')
    return x
col = u'время выдержки в закалочной ванне, сек.'
df_modes[col] = df_modes[col].apply(clean_time_bath)
df_valid[col] = df_valid[col].apply(clean_time_bath)

In [49]:
# Удаляем лишнии столбцы из датафрейма
l2del_modes = [
#     'Дата термообработки', 
    u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка',
      '2 зона уставка закалка', '2 зона фактическая температура закалка', '2 зона фактическая температура отпуск', 
      '4 зона фактическая температура отпуск', 'кол-во, шт.'
]

l2del_valid =[
#     'Дата термообработки', 
    u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка', 'кол-во, шт.', 
    'масса\nтн']

def del_cols_df(df, l2del):
    df[u'№ плавки'] = df[u'плавка']
    df = df.drop(l2del, axis = 1)
    return df

df_modes = del_cols_df(df_modes, l2del_modes)
df_valid = del_cols_df(df_valid, l2del_valid)
print(df_modes.shape, df_valid.shape)

(129047, 33) (8958, 22)


In [50]:
# Объединение марок стали
def rep_steel(x):
    x = str(x)
    x = x.replace('092ГС','09Г2С')
    x = x.replace('13 ХФА','13ХФА')
    x = x.replace('13хфа','13ХФА')
    x = x.replace('13ХФА ','13ХФА')
    x = x.replace('13хфа-3','13ХФА-3')
    x = x.replace('15Х13H2MБ','15Х13H2МБ')
    x = x.replace('15х5мфбч','15Х5МФБЧ')
    x = x.replace('18х3мфб','18Х3МФБ')
    x = x.replace('18Х3МФБ ','18Х3МФБ')
    x = x.replace('18х3ХМФБ','18Х3МФБ')
    x = x.replace('18хзмфб','18Х3МФБ')
    x = x.replace('18хмфб','18ХМФБ')
    x = x.replace('18ХМФБ ','18ХМФБ')
    x = x.replace('20А простая','20А-пр')
    x = x.replace('20А-ПР','20А-пр')
    x = x.replace('20 А','20А')    
    x = x.replace('20A','20А') 
    x = x.replace('20А ','20А')
    x = x.replace('20 АУ','20А-У')
    x = x.replace('20АУ','20А-У')
    x = x.replace('30Г2 ','30Г2')
    x = x.replace('30Г2.','30Г2')
    x = x.replace('30г2-2','30Г2-2')
    x = x.replace('30г2ф','30Г2Ф')
    x = x.replace('30г2','30Г2')
    x = x.replace('30хгма','30ХГМА')
    x = x.replace('30ХГМА1','30ХГМА-1')
    x = x.replace('30хма','30ХМА')
    x = x.replace('32г1','32Г1')
    x = x.replace('32г2','32Г2')
    x = x.replace('32х1мф-1','32Х1МФ-1')
    x = x.replace('32ха','32ХА')
    x = x.replace('37г2ф','37Г2Ф')
    x = x.replace('37г2Ф','37Г2Ф')
    x = x.replace('37Г2ф','37Г2Ф')
    x = x.replace('37Г2Ф ','37Г2Ф')
    x = x.replace('38Г2С ','38Г2С')
    x = x.replace('38г2с-4','38Г2С-4')
    x = x.replace('38Г2-С-9','38Г2С-9')
    x = x.replace('30Г2 ', '30Г2')
    return x

df_modes['марка стали'] = df_modes['марка стали'].apply(rep_steel)
df_valid['марка стали'] = df_valid['марка стали'].apply(rep_steel)

In [51]:
ls_vtr = [
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск'
]
# Очистка всех столбцов с температурами по втр 
def clean_vtr(x):
    x = str(x)
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('-','')
    x = x.replace('50/*','')
    x = x.replace('нет','')
    x = x.replace('НЕТ','')
    x = x.replace('н/д','')
    return x


def clean_all(df, ls, apply_f):
    """Применяет к столбцам из списка ls датафрейма df метод apply_f"""
    for col in ls:
        df[col] = df[col].apply(apply_f)
    return df

df_modes = clean_all(df_modes, ls_vtr, clean_vtr)
df_valid = clean_all(df_valid, ls_vtr, clean_vtr)

In [52]:
ls_nd = [
    '3 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    '1 зона уставка закалка',
    '1 зона фактическая температура закалка',
    '3 зона уставка закалка',
    '3 зона фактическая температура закалка',
    'Температура трубы на выгрузке из печи закалки',
    'Расход воды на закалочный спрейер, м3',
    't˚ C воды в закалочной ванне',
    '1 зона фактическая температура отпуск',
    '3 зона фактическая температура отпуск',
    '5 зона фактическая температура отпуск',
    'Температура трубы на выгрузке из печи отпуска'
]
# Чистим столбцы от н/д
def rep_nd(x):
    x = str(x)
    x = x.replace('н/д','')
    x = x.replace('.0', '')
    x = x.replace('НЕТ', '')
    x = x.replace('***', '')
#     x = x.replace('охлаждение','')
    return x

df_modes = clean_all(df_modes, ls_nd, rep_nd)
df_valid = clean_all(df_valid, ls_nd[:-11], rep_nd)

In [53]:
# Чистим время выдержки в закалочной ванне, сек.
def rep_time(x):
    x = str(x)
    x = x.replace('nan','')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')
    x = x.replace('НОРМ','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('40-50','45')
    x = x.replace('сек','')
    x = x.replace('с','')
    x = x.replace(',','.')
    x = x.replace('11cек', '')
    x = x.replace('HET', '')
    return x


col ='время выдержки в закалочной ванне, сек.'
df_modes[col] = df_modes[col].apply(rep_time)
df_valid[col] = df_valid[col].apply(rep_time)

In [54]:
def rep_tempr(x):
    x = str(x)
    x = x.replace('******','')
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('*','')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция', '')
    x = x.replace('\\','/')
    x = x.replace('-','/')
    x = x.replace('<', '')
    x = x.replace('>', '')
    x = x.replace('Доотпуск', 'доотпуск')
    x = x.replace('до', '')
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('БизнесТренд', '')
    x = x.replace('ВОЗДУХ', '')
    x = x.replace('Воздух', '')
    x = x.replace('воэдух', '')
    x = x.replace('высадка', '')
    x = x.replace('закалка', '')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('нет', '')
    x = x.replace('НОРМ', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('воздух', '')
    x = x.replace('61702', '')
    x = x.replace('опытные.','')
    x = x.replace('на е','')
    x = x.replace('норм/ия','')
    x = x.replace('нормал/ция','')
    x = x.replace('АЛ/ЦИЯ','')
    x = x.replace(',','.')
    y = x.split('/')
    if len(y)==1:
        return x
    fl = True
    tmpy= []
    for i in y:
        if len(i) == 4:
            y.append(i[0:2])
            y.append(i[2:4])
            y.remove(i)
    for i in y:
        try:
            tmpy.append(float(i))
        except:
            tmpy.append(i)
            fl = False
    if(fl):
        y = tmpy   
        x = np.mean(y)
    if x=='0':
        x=''
    return x


def clean_and_tofloat(df, ls, apply_f_ls):
    
    for col, apply_f in zip(ls,apply_f_ls):
        df[col] = df[col].apply(apply_f)
    df.drop(df[df[col]=='отпуск'].index, inplace = True, axis = 0)
    df[col] = df[col].apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)
    return df

col = 't˚ C трубы после спреера'
df_modes = clean_and_tofloat(df_modes, [col], [rep_tempr])
df_valid = clean_and_tofloat(df_valid, [col], [rep_tempr])
# df_modes[col] = df_modes[col].apply(rep_tempr)
# df_modes.drop(df_modes[df_modes[col]=='отпуск'].index, inplace = True, axis = 0)
# df_modes[col] = df_modes[col].apply(
#                 lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)

In [55]:
col = 'масса\nтн'
df_valid = df_valid.rename(columns={'масса\nтн.1': col})
df_valid = clean_and_tofloat(df_valid, [col], [rep_mass])
# df_valid['масса\nтн'].value_counts().sort_index()

In [57]:
def rep_mass(x):
    x = str(x)
    x = x.replace('-','')
    x = x.replace(',','.')
    x = x.replace('/','')
    x = x.replace('73.02х5.51','')
    x = x.replace('4,,640','')
    x = x.replace('3.000.','3.0')
    x = x.replace('.1.440','1.44')
    x = x.replace('01б','0.16')
    return x

col = 'масса\nтн'
df_modes = clean_and_tofloat(df_modes, [col], [rep_mass])

In [58]:
def rep_tempr_bath(x):
    x = str(x)
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('бизнес тренд', '')
    x = x.replace('высадка.', '')
    x = x.replace('высадка', '')
    x = x.replace('ВЫСАДКА', '')
    x = x.replace('Высадка', '')
    x = x.replace('высадка', '') 
    x = x.replace('доотпуск.', 'отпуск')
    x = x.replace('доотпуск', 'отпуск')
    x = x.replace('Доотпуск', 'отпуск')
    x = x.replace('отпуск.', 'отпуск')
    x = x.replace('на отпуск', 'отпуск')
    x = x.replace('в малахи', '')
    x = x.replace('ДООТПУСК', 'отпуск')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('НК 8500', '')
    x = x.replace('НК 9000', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('нормал-ция', '')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция.', '')
    x = x.replace('норм-ция', '')
    x = x.replace('нор-ция', '')
    x = x.replace('нор/ция', '')
    x = x.replace('опытные.', '')
    x = x.replace('ОПЫТНЫЕ', '')
    x = x.replace('повторная', '')
    x = x.replace('повтор', '')
    x = x.replace('ПОВТОР', '')
    x = x.replace('Самара', '')
    x = x.replace('СИТЦ', '')
    x = x.replace('Трэнд', '')
    x = x.replace('ЧПТЗ', '')
    x = x.replace('ЧТПЗ', '')
    x = x.replace('30 СЕК.', '30')
    x = x.replace('2/я зак/ка', '')
    x = x.replace('ТРЕНД', '')
    x = x.replace('ПЛ.Р. Ц42/№10','')
    x = x.replace('Челябинск','')
    x = x.replace('нормал/ция', '')
    x = x.replace('HET', '')
    x = x.replace('J55', '')
    x = x.replace('/', '')
    if x=='0':
        x=''
    if x=='.':
        x=''
    return x

col = 't˚ C трубы после ванны'
df_modes = clean_and_tofloat(df_modes, [col, col], [rep_tempr, rep_tempr_bath])
df_valid = clean_and_tofloat(df_valid, [col, col], [rep_tempr, rep_tempr_bath])
# df_modes[col] = df_modes[col].apply(rep_tempr)
# df_modes[col] = df_modes[col].apply(rep_tempr_bath)
# df_modes.drop(df_modes[df_modes[col]=='отпуск'].index, inplace = True, axis = 0)
# df_modes[col] = df_modes[col].apply(
#                 lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)
# df_modes[col].to_excel('check_col.xlsx')

In [59]:
print(df_modes.shape, df_valid.shape)

(128537, 33) (8958, 22)


In [60]:
df_modes.drop(df_modes[df_modes['шаг балок закалочная печь, сек'] == '0'].index, inplace=True, axis=0)
df_modes.drop(df_modes[df_modes['шаг балок отпускная печь, сек'] == '0'].index, inplace=True, axis=0)
df_valid.drop(df_valid[df_valid['шаг балок закалочная печь, сек'] == '0'].index, inplace=True, axis=0)
df_valid.drop(df_valid[df_valid['шаг балок отпускная печь, сек'] == '0'].index, inplace=True, axis=0)
print(df_modes.shape, df_valid.shape)

(126113, 33) (8849, 22)


In [61]:
ls_mean_modes = df_modes.columns[3:-3]
ls_mean_valid = df_valid.columns[3:-3]

def get_mean(df, ls_mean):
    """Усредняет по [номеру плавки, партиям] и возвращает полученный датафрейм """
    for col in ls_mean:
        if col in df:
            try:
                df[col] = df.groupby(['№ плавки', '№ партии'])[col].transform(
                    lambda x: x.apply(
                    lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float).mean())
            except:
                print(col)
    return df

In [62]:
df_modes['№ партии'] = df_modes['№ партии'].astype(str).str.lower()
df_modes['№ плавки'] = df_modes['№ плавки'].astype(str).str.lower()
df_valid['№ партии'] = df_valid['№ партии'].astype(str).str.lower()
df_valid['№ плавки'] = df_valid['№ плавки'].astype(str).str.lower()

In [63]:
print(df_modes.shape, df_valid.shape)
print(df_modes['№ партии'].describe()[1])#количество уникальных значений по партии, итоговое должно быть чуть больше
print(df_valid['№ партии'].describe()[1])#количество уникальных значений по партии, итоговое должно быть чуть больше

(126113, 33) (8849, 22)
28600
1700


In [64]:
col = 'время выдержки в закалочной ванне, сек.'
df_valid[col].value_counts()

      8376
25     264
20     133
30      63
40      12
10       1
Name: время выдержки в закалочной ванне, сек., dtype: int64

In [65]:
# Усреднение по номеру плавки, партиям работает долго, ждем и пьем чаёк..
tmp_modes = get_mean(df_modes, ls_mean_modes)
tmp_modes.shape

Дата термообработки


(126113, 33)

In [66]:
tmp_valid = get_mean(df_valid, ls_mean_valid)
tmp_valid.shape

Дата термообработки


(8849, 22)

In [67]:
full_ls_modes = list(df_modes.columns)
full_ls_modes = full_ls_modes[-1:]+full_ls_modes[0:-1]

full_ls_valid = list(df_valid.columns)
full_ls_valid = full_ls_valid[-1:]+full_ls_valid[0:-1]

In [68]:
tmp_modes = tmp_modes[full_ls_modes]
tmp_modes = tmp_modes.drop_duplicates()

tmp_valid = tmp_valid[full_ls_valid]
tmp_valid = tmp_valid.drop_duplicates()
print(tmp_modes.shape, tmp_valid.shape)

(33174, 33) (2046, 22)


In [69]:
tmp_modes['2 зона по ВТР закалка'] = tmp_modes['2 зона по ВТР закалка'].apply(lambda x: x*10 if x<100 else x)
tmp_modes.shape

(33174, 33)

In [70]:
def th2f(x):
    if x=='73.02*5.51':
        return 5.51
    if (x==' ') | (x==''):
        return None
    return float(x)
# tmp_modes[tmp_modes[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values('№ партии')
tmp_modes['толщина стенки'] = tmp_modes['толщина стенки'].apply(th2f)
tmp_valid['толщина стенки'] = tmp_valid['толщина стенки'].apply(th2f)

In [71]:
tmp_valid.shape

(2046, 22)

In [72]:
tmp_modes = tmp_modes.groupby(['№ плавки', '№ партии']).first()
tmp_modes = tmp_modes.reset_index()
print(tmp_modes.shape)

(29255, 33)


In [73]:
tmp_valid = tmp_valid.groupby(['№ плавки', '№ партии']).first()
tmp_valid = tmp_valid.reset_index()
print(tmp_valid.shape)

(1722, 22)


In [74]:
tmp_modes.to_excel('all_modes_clean.xlsx')
tmp_valid.to_excel('valid_modes_clean.xlsx')

In [75]:
tmp_modes.head()

,№ плавки,№ партии,"НД (гост, ту)",марка стали,Дата термообработки,1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,3 зона уставка закалка,3 зона фактическая температура закалка,Температура трубы на выгрузке из печи закалки,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,"Расход воды на закалочный спрейер, м3","время выдержки в закалочной ванне, сек.",t˚ C воды в закалочной ванне,t˚ C трубы после ванны,1 зона ВТР и уставка отпуск,1 зона фактическая температура отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска,масса тн,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки
0,0039,701354,ТУ 20,09Г2С,25.01.2017,900.0,900.0,899.666667,900.0,910.0,900.000000,900.166667,897.666667,0.46,47.5,1617.833333,NaN,20.0,NaN,682.500000,681.500000,680.0,673.0,673.333333,668.0,656.000000,655.833333,646.666667,3.910000,30.0,30.0,114,6.0
1,0043,701351,ТУ 20,09Г2С,25.01.2017,900.0,900.0,899.166667,900.0,910.0,902.666667,901.833333,897.333333,0.46,47.5,1625.666667,NaN,20.0,NaN,688.333333,688.166667,685.0,679.0,678.833333,673.0,662.666667,662.500000,649.666667,4.513333,30.0,30.0,114,6.0
2,0043,701353,ТУ 20,09Г2С,25.01.2017,900.0,900.0,898.500000,900.0,910.0,900.000000,899.750000,895.500000,0.46,47.5,1639.750000,NaN,20.0,NaN,685.000000,686.250000,685.0,673.0,672.250000,673.0,656.000000,656.000000,650.250000,3.817500,30.0,30.0,114,6.0
3,013495,206349,ТУ 3,30Г2,23.06.2012,850.0,850.0,848.800000,850.0,860.0,848.000000,847.800000,849.000000,0.54,54.4,2066.000000,NaN,27.0,NaN,560.000000,561.400000,560.0,555.0,556.000000,555.0,547.000000,546.800000,561.200000,3.607800,28.0,27.0,88.9,6.5
4,013495,208207,ТУ 3,30Г2,14.08.2012,850.0,850.0,846.000000,850.0,860.0,845.000000,849.000000,NaN,0.51,55.0,NaN,NaN,26.0,NaN,560.000000,559.000000,560.0,555.0,553.000000,555.0,547.000000,547.000000,NaN,0.500000,29.0,27.0,88.9,6.5
